# TITIM Complete Experiment Notebook

**Based on README.md and all script files — Generated for Jupyter Notebook implementation**

---

## TITIM: Training-Inference Trigger Intensity Mismatch in Backdoor Attacks

**USENIX Security '25 Paper Implementation**

This notebook implements the complete TITIM experiment pipeline based on the paper **"Revisiting Training-Inference Trigger Intensity in Backdoor Attacks"**.

### 🎯 Research Question
**How does the mismatch between training and inference trigger intensities affect backdoor attack effectiveness?**

### 📋 Experiment Pipeline
1. **Environment Setup** — Install dependencies and check hardware  
2. **Dataset Preparation** — Download clean datasets (CIFAR-10, MNIST, GTSRB)  
3. **Poisoned Data Generation** — Create datasets with varying trigger intensities (0.1–1.0)  
4. **Model Training** — Train ResNet18 models on different intensity datasets  
5. **Cross-Intensity Testing** — Test models on different intensity datasets (10×10 matrix)  
6. **Visualization & Analysis** — Generate heatmaps and analyze results  
7. **Defense Evaluation** — Test ABL, Neural Cleanse, STRIP, etc.  
8. **Mixed Intensity Experiments** — Advanced experiments with mixed intensities

### 🔧 Hardware Requirements
- **Minimum**: 8GB RAM, 2GB NVIDIA GPU (for ResNet18 on CIFAR-10)  
- **Recommended**: 128GB RAM, 24GB NVIDIA GPU (for full experiments)  
- **OS**: Linux recommended (Ubuntu 22.04.4)


In [ ]:
# === CELL 2: Environment Setup and Dependencies ===
# Import libraries and setup environment
import os
import sys
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.display import display, HTML
import warnings
import time
from datetime import datetime
warnings.filterwarnings('ignore')

# Change to TITIM-main directory
os.chdir('TITIM-main')
print("📂 Current working directory:", os.getcwd())

# Check Python version
print("🐍 Python version:", sys.version)

# Check GPU availability
try:
    import torch
    print(f"🔥 PyTorch version: {torch.__version__}")
    print(f"🎮 CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🎯 GPU count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
    else:
        print("⚠️  No GPU detected - CPU only mode")
except ImportError:
    print("❌ PyTorch not installed")

# Define helper function
def run_command(cmd, show_output=True, background=False):
    """Execute shell command and return result"""
    print(f"⚡ Executing: {cmd}")
    if background:
        subprocess.Popen(cmd, shell=True)
        return None
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if show_output and result.stdout:
        print("📤 Output:", result.stdout.strip())
    if result.stderr and result.returncode != 0:
        print("❌ Error:", result.stderr.strip())
    return result


In [ ]:
# === CELL 3: Dependency Check ===
print("🔍 Checking dependencies...")

required_packages = [
    'torch', 'torchvision', 'numpy', 'pandas', 'matplotlib', 
    'sklearn', 'cv2', 'seaborn', 'tqdm', 'PIL'
]

missing_packages = []
for package in required_packages:
    try:
        if package == 'cv2':
            import cv2
        elif package == 'sklearn':
            import sklearn
        elif package == 'PIL':
            from PIL import Image
        else:
            __import__(package)
        print(f"✅ {package}: Installed")
    except ImportError:
        print(f"❌ {package}: Missing")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️  Missing packages: {missing_packages}")
    print("Please run: pip install -r requirements.txt")
else:
    print("\n🎉 All dependencies satisfied!")

# Check if we're in virtual environment
if 'titim' in sys.executable.lower():
    print("🌐 Running in TITIM virtual environment")
else:
    print("⚠️  Not in virtual environment - consider using one")


In [ ]:
# === CELL 4: Dataset Preparation ===
# Download clean datasets following scripts/get_clean_datasets.sh
print("📦 Downloading clean datasets...")

datasets = ['cifar10', 'mnist', 'gtsrb']  # Excluding celeba8 for faster execution
success_count = 0

for dataset in datasets:
    print(f"\n📥 Getting {dataset.upper()} dataset...")
    cmd = f"python utils/scripts/dataset_get_clean.py --dataset {dataset}"
    result = run_command(cmd, show_output=False)
    
    if result.returncode == 0:
        print(f"✅ {dataset.upper()} downloaded successfully")
        success_count += 1
    else:
        print(f"❌ {dataset.upper()} download failed")
        if result.stderr:
            print(f"Error: {result.stderr}")

print(f"\n📊 Dataset download summary: {success_count}/{len(datasets)} successful")

# Verify dataset structure
print("\n🔍 Verifying dataset structure...")
data_dir = "data"
if os.path.exists(data_dir):
    for dataset in datasets:
        dataset_path = os.path.join(data_dir, dataset, "clean")
        if os.path.exists(dataset_path):
            files = os.listdir(dataset_path)
            print(f"✅ {dataset}: {files}")
        else:
            print(f"❌ {dataset}: Directory not found")
else:
    print("❌ Data directory not found")


In [ ]:
# === CELL 5: Poisoned Dataset Generation ===
# Generate poisoned datasets with varying trigger intensities
# Following scripts/inject/inject_badnets_square.sh

print("🦠 Generating poisoned datasets with BadNets (Square) attack...")

# Experiment parameters
dataset = 'cifar10'  # Focus on CIFAR-10 for main experiment
target = 0
trigger = 'badnets'
ratio = 0.05  # 5% poisoning ratio
intensities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

print(f"🎯 Target class: {target}")
print(f"🦠 Poisoning ratio: {ratio}")
print(f"🎛️  Trigger intensities: {intensities}")
print(f"🔲 Trigger type: {trigger} (4x4 square)")

# Generate datasets for each intensity
success_count = 0
start_time = time.time()

for i, mr in enumerate(intensities):
    print(f"\n[{i+1}/{len(intensities)}] 🎛️  Generating intensity {mr} dataset...")
    
    cmd = (f"python inject.py --dataset {dataset} --target {target} "
           f"--ratio {ratio} --trigger {trigger} --mr {mr} "
           f"--block_size 4 --fixed --split_val test")
    
    result = run_command(cmd, show_output=False)
    
    if result.returncode == 0:
        print(f"✅ Intensity {mr}: SUCCESS")
        success_count += 1
    else:
        print(f"❌ Intensity {mr}: FAILED")
        if result.stderr:
            print(f"   Error: {result.stderr[:200]}...")

elapsed_time = time.time() - start_time
print(f"\n⏱️  Generation completed in {elapsed_time:.1f}s")
print(f"📊 Success rate: {success_count}/{len(intensities)} ({100*success_count/len(intensities):.1f}%)")

# Verify generated datasets
print("\n🔍 Verifying generated datasets...")
poisoned_datasets = []
data_dir = f"data/{dataset}"
if os.path.exists(data_dir):
    for item in os.listdir(data_dir):
        if item.startswith('badnets_b4_bn3_ppt1_mr'):
            poisoned_datasets.append(item)
            dataset_path = os.path.join(data_dir, item)
            files = os.listdir(dataset_path)
            print(f"✅ {item}: {len(files)} files")

print(f"\n📈 Total poisoned datasets: {len(poisoned_datasets)}")


In [ ]:
# === CELL 6: Backdoor Model Training ===
# Train backdoor models following scripts/train/train_badnets_square.sh
print("🎓 Training backdoor models...")

# Training parameters
model = 'resnet18'
epochs = 100
batch_size = 128
learning_rate = 1e-2
optimizer = 'adam'
gpu = 0

print(f"🏗️  Model architecture: {model}")
print(f"🔄 Training epochs: {epochs}")
print(f"📦 Batch size: {batch_size}")
print(f"📈 Learning rate: {learning_rate}")
print(f"⚙️  Optimizer: {optimizer}")

# Create logs directory
os.makedirs("logs", exist_ok=True)
log_file = "logs/badnets.tsv"

# Train models for each intensity
success_count = 0
start_time = time.time()

for i, mr in enumerate(intensities):
    print(f"\n[{i+1}/{len(intensities)}] 🎓 Training model for intensity {mr}...")
    
    subset = f"badnets_b4_bn3_ppt1_mr{mr}_{ratio}"
    cmd = (f"python train.py --gpu {gpu} --model {model} --dataset {dataset} "
           f"--subset {subset} --epochs {epochs} --bs {batch_size} "
           f"--lr {learning_rate} --optimizer {optimizer} --log {log_file} "
           f"--mode cover --split_val test --disable_prog")
    
    model_start = time.time()
    result = run_command(cmd, show_output=False)
    model_time = time.time() - model_start
    
    if result.returncode == 0:
        print(f"✅ Intensity {mr}: SUCCESS ({model_time:.1f}s)")
        success_count += 1
    else:
        print(f"❌ Intensity {mr}: FAILED")
        if result.stderr:
            print(f"   Error: {result.stderr[:200]}...")

total_time = time.time() - start_time
print(f"\n⏱️  Training completed in {total_time:.1f}s ({total_time/60:.1f} minutes)")
print(f"📊 Success rate: {success_count}/{len(intensities)} ({100*success_count/len(intensities):.1f}%)")

# Verify trained models
print("\n🔍 Verifying trained models...")
trained_models = []
res_dir = "res"
if os.path.exists(res_dir):
    for item in os.listdir(res_dir):
        if 'resnet18' in item and 'badnets' in item:
            trained_models.append(item)
            model_path = os.path.join(res_dir, item)
            files = os.listdir(model_path)
            print(f"✅ {item}: {len(files)} files")

print(f"\n📈 Total trained models: {len(trained_models)}")


In [ ]:
# === CELL 7: Cross-Intensity Testing (Core Experiment) ===
# Cross-intensity testing following scripts/inference/crosstest_badnets_square.sh
print("🎯 Starting cross-intensity testing (CORE EXPERIMENT)...")
print("This tests how models trained on one intensity perform on different intensities.")

# Setup test log file
test_log_file = "logs/cross_badnets_square.tsv"
os.makedirs("logs", exist_ok=True)

# Initialize log file with header
with open(test_log_file, 'w') as f:
    f.write("name\tasr\n")

print(f"📝 Results will be saved to: {test_log_file}")

# Calculate total number of tests
total_tests = len(intensities) * len(intensities)
print(f"📊 Total tests to perform: {total_tests} (10x10 matrix)")

# Perform cross-intensity testing
success_count = 0
current_test = 0
start_time = time.time()

for train_mr in intensities:
    for test_mr in intensities:
        current_test += 1
        print(f"\n[{current_test}/{total_tests}] 🎯 Testing: Train={train_mr}, Test={test_mr}")
        
        # Construct paths and names
        model_dir = f"{model}_{dataset}_badnets_b4_bn3_ppt1_mr{train_mr}_{ratio}_e{epochs}"
        test_subset = f"badnets_b4_bn3_ppt1_mr{test_mr}_{ratio}"
        test_name = f"{model}_{dataset}_badnets_b4_bn3_ppt1_mr{train_mr}_{ratio}_e{epochs}_tmr{test_mr}"
        
        cmd = (f"python evaluate.py --gpu {gpu} --model_dir {model_dir} "
               f"--dataset {dataset} --subset {test_subset} --split test "
               f"--log {test_log_file} --name {test_name}")
        
        result = run_command(cmd, show_output=False)
        
        if result.returncode == 0:
            print(f"✅ SUCCESS")
            success_count += 1
        else:
            print(f"❌ FAILED")
            if result.stderr:
                print(f"   Error: {result.stderr[:100]}...")

elapsed_time = time.time() - start_time
print(f"\n⏱️  Cross-intensity testing completed in {elapsed_time:.1f}s ({elapsed_time/60:.1f} minutes)")
print(f"📊 Success rate: {success_count}/{total_tests} ({100*success_count/total_tests:.1f}%)")

# Load and preview results
if os.path.exists(test_log_file):
    print(f"\n📈 Loading results from {test_log_file}...")
    results_df = pd.read_csv(test_log_file, sep='\t')
    print(f"📊 Total results: {len(results_df)}")
    
    if len(results_df) > 0:
        print("\n🔍 Preview of results:")
        display(results_df.head(10))
        
        # Basic statistics
        asr_values = pd.to_numeric(results_df['asr'], errors='coerce')
        print(f"\n📊 ASR Statistics:")
        print(f"   📈 Max ASR: {asr_values.max():.3f}")
        print(f"   📉 Min ASR: {asr_values.min():.3f}")
        print(f"   📊 Mean ASR: {asr_values.mean():.3f}")
        print(f"   📏 Std ASR: {asr_values.std():.3f}")
else:
    print("❌ Results file not found")


In [ ]:
# === CELL 8: Results Visualization & Analysis ===
# Generate official heatmap using utils/scripts/draw_heatmap.py
print("📊 Generating visualization and analysis...")

# Use official heatmap script
heatmap_name = "resnet18_cifar10_badnets_b4_bn3_ppt1_mrx_0.05"
heatmap_cmd = f"python utils/scripts/draw_heatmap.py --name {heatmap_name} --N 10"
print(f"🎨 Generating official heatmap: {heatmap_name}")

result = run_command(heatmap_cmd, show_output=False)
if result.returncode == 0:
    print("✅ Official heatmap generated successfully")
else:
    print("❌ Official heatmap generation failed")
    if result.stderr:
        print(f"Error: {result.stderr}")

# Check generated files
heatmap_dir = "heatmaps"
if os.path.exists(heatmap_dir):
    files = os.listdir(heatmap_dir)
    print(f"📁 Generated heatmap files: {files}")
else:
    print("📁 Heatmap directory not found")


In [ ]:
# === CELL 9: Custom Analysis and Visualization ===
# Custom analysis and visualization
if os.path.exists(test_log_file):
    print("🔬 Performing custom analysis...")
    
    # Load results
    results_df = pd.read_csv(test_log_file, sep='\t')
    
    # Extract training and testing intensities
    results_df['train_mr'] = results_df['name'].str.extract(r'mr(\d+\.\d+)_0\.05_e100_tmr')
    results_df['test_mr'] = results_df['name'].str.extract(r'tmr(\d+\.\d+)')
    
    # Convert to numeric
    results_df['train_mr'] = pd.to_numeric(results_df['train_mr'])
    results_df['test_mr'] = pd.to_numeric(results_df['test_mr'])
    results_df['asr'] = pd.to_numeric(results_df['asr'], errors='coerce')
    
    # Create pivot table for heatmap
    pivot_table = results_df.pivot(index='train_mr', columns='test_mr', values='asr')
    
    # Generate custom heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='YlOrRd', 
                cbar_kws={'label': 'Attack Success Rate (ASR)'},
                square=True, linewidths=0.5)
    plt.title('TITIM: Cross-Intensity Attack Success Rate\n(Training Intensity vs Testing Intensity)', 
              fontsize=16, fontweight='bold')
    plt.xlabel('Test Trigger Intensity', fontsize=14)
    plt.ylabel('Training Trigger Intensity', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Diagonal analysis (Training = Testing intensity)
    diagonal_asr = []
    for mr in intensities:
        if mr in pivot_table.index and mr in pivot_table.columns:
            diagonal_asr.append(pivot_table.loc[mr, mr])
        else:
            diagonal_asr.append(np.nan)
    
    plt.figure(figsize=(10, 6))
    plt.plot(intensities, diagonal_asr, 'bo-', linewidth=3, markersize=10, label='Matched Intensity')
    plt.xlabel('Trigger Intensity (Training = Testing)', fontsize=12)
    plt.ylabel('Attack Success Rate', fontsize=12)
    plt.title('ASR vs Trigger Intensity (Matched Training-Testing)', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print("\n🔍 Diagonal Analysis (Training = Testing intensity):")
    for i, mr in enumerate(intensities):
        if not np.isnan(diagonal_asr[i]):
            print(f"   Intensity {mr}: ASR = {diagonal_asr[i]:.3f}")
    
    # Mismatch analysis
    results_df['intensity_mismatch'] = abs(results_df['train_mr'] - results_df['test_mr'])
    mismatch_corr = results_df['intensity_mismatch'].corr(results_df['asr'])
    
    print(f"\n📊 Key Findings:")
    print(f"   🔗 Intensity mismatch vs ASR correlation: {mismatch_corr:.3f}")
    
    # Best and worst cases
    max_idx = results_df['asr'].idxmax()
    min_idx = results_df['asr'].idxmin()
    best_case = results_df.loc[max_idx]
    worst_case = results_df.loc[min_idx]
    
    print(f"   🏆 Highest ASR: {best_case['asr']:.3f} (Train: {best_case['train_mr']}, Test: {best_case['test_mr']})")
    print(f"   📉 Lowest ASR: {worst_case['asr']:.3f} (Train: {worst_case['train_mr']}, Test: {worst_case['test_mr']})")
    
else:
    print("❌ No results file found for analysis")


In [ ]:
# === CELL 10: Defense Evaluation ===
# Test defense methods following scripts/defenses/
print("🛡️  Testing backdoor defense methods...")

defense_methods = {
    'abl': 'Activation-based Backdoor Learning',
    'nc': 'Neural Cleanse', 
    'strip': 'STRIP',
    'ss': 'Spectral Signature'
}

print("Available defense methods:")
for method, description in defense_methods.items():
    print(f"   🛡️  {method.upper()}: {description}")

# Test Neural Cleanse (fastest to demonstrate)
print(f"\n🧪 Testing Neural Cleanse defense...")
defense_cmd = f"bash scripts/defenses/defense_nc.sh {gpu} {dataset} {model}"
result = run_command(defense_cmd, show_output=False)

if result.returncode == 0:
    print("✅ Neural Cleanse defense test completed")
else:
    print("❌ Neural Cleanse defense test failed")
    if result.stderr:
        print(f"Error: {result.stderr[:200]}...")

# Check defense results
defense_log = "logs/defense_nc.csv"
if os.path.exists(defense_log):
    print(f"\n📊 Defense results saved to: {defense_log}")
    try:
        defense_results = pd.read_csv(defense_log)
        print(f"📈 Defense test results: {len(defense_results)} entries")
        if len(defense_results) > 0:
            display(defense_results.head())
    except Exception as e:
        print(f"Error reading defense results: {e}")
else:
    print("❌ Defense results file not found")

print("\n💡 To test other defenses, run the corresponding scripts:")
print("   bash scripts/defenses/defense_abl.sh 0 cifar10 resnet18")
print("   bash scripts/defenses/defense_strip.sh 0 cifar10 resnet18")
print("   bash scripts/defenses/defense_ss.sh 0 cifar10 resnet18")


In [ ]:
# === CELL 11: Mixed Intensity Experiments ===
# Mixed intensity experiments following scripts/mixtest/
print("🎭 Running mixed intensity experiments...")
print("This tests models trained with mixed trigger intensities.")

# Mixed intensity parameters
mixmr = 0.1  # Base intensity for mixing
print(f"🎛️  Base mixing intensity: {mixmr}")

# Run mixed intensity scripts
mix_scripts = [
    ("inject_mix.sh", "Generate mixed intensity datasets"),
    ("train_mix.sh", "Train mixed intensity models"),
    ("crosstest_mix.sh", "Cross-test mixed models"),
    ("draw_mix.sh", "Generate mixed intensity heatmap")
]

for i, (script, description) in enumerate(mix_scripts):
    print(f"\n[{i+1}/{len(mix_scripts)}] 🎭 {description}...")
    
    if script == "train_mix.sh":
        cmd = f"bash scripts/mixtest/{script} {gpu} {dataset} {model} {ratio} {mixmr}"
    elif script == "crosstest_mix.sh":
        cmd = f"bash scripts/mixtest/{script} {gpu} {dataset} {model} {ratio} {mixmr}"
    else:
        cmd = f"bash scripts/mixtest/{script}"
    
    result = run_command(cmd, show_output=False)
    
    if result.returncode == 0:
        print(f"✅ {script}: SUCCESS")
    else:
        print(f"❌ {script}: FAILED")
        if result.stderr:
            print(f"   Error: {result.stderr[:200]}...")

# Check mixed intensity results
mix_log_file = f"logs/cross_badnets_mixmr{mixmr}_{ratio}_{model}.tsv"
if os.path.exists(mix_log_file):
    print(f"\n📊 Mixed intensity results: {mix_log_file}")
    mix_results = pd.read_csv(mix_log_file, sep='\t')
    print(f"📈 Mixed intensity test results: {len(mix_results)} entries")
    if len(mix_results) > 0:
        display(mix_results.head())
else:
    print(f"❌ Mixed intensity results not found: {mix_log_file}")

# Check for mixed intensity heatmap
mix_heatmap = f"heatmaps/cross_badnets_mixmr{mixmr}_{ratio}_{model}.pdf"
if os.path.exists(mix_heatmap):
    print(f"✅ Mixed intensity heatmap generated: {mix_heatmap}")
else:
    print(f"❌ Mixed intensity heatmap not found: {mix_heatmap}")


In [ ]:
# === CELL 12: Experiment Summary & Final Analysis ===
# Comprehensive experiment summary
print("📋 TITIM EXPERIMENT SUMMARY")
print("="*50)

# Check completion status
experiment_stages = {
    "📦 Dataset Preparation": os.path.exists("data/cifar10/clean"),
    "🦠 Poisoned Data Generation": len([d for d in os.listdir("data/cifar10") if d.startswith('badnets')]) >= 5 if os.path.exists("data/cifar10") else False,
    "🎓 Model Training": len([d for d in os.listdir("res") if 'resnet18' in d and 'badnets' in d]) >= 5 if os.path.exists("res") else False,
    "🎯 Cross-Intensity Testing": os.path.exists("logs/cross_badnets_square.tsv"),
    "📊 Visualization": os.path.exists("heatmaps") and len(os.listdir("heatmaps")) > 0 if os.path.exists("heatmaps") else False,
    "🛡️  Defense Evaluation": os.path.exists("logs/defense_nc.csv"),
    "🎭 Mixed Intensity Tests": os.path.exists(f"logs/cross_badnets_mixmr{mixmr}_{ratio}_{model}.tsv")
}

print("\n🏁 Experiment Completion Status:")
completed_stages = 0
for stage, status in experiment_stages.items():
    status_icon = "✅" if status else "❌"
    print(f"   {status_icon} {stage}")
    if status:
        completed_stages += 1

completion_rate = completed_stages / len(experiment_stages) * 100
print(f"\n📊 Overall completion: {completed_stages}/{len(experiment_stages)} ({completion_rate:.1f}%)")

# File structure summary
print("\n📁 Generated File Structure:")
directories = ['data', 'res', 'logs', 'heatmaps']
for directory in directories:
    if os.path.exists(directory):
        files = os.listdir(directory)
        print(f"   📂 {directory}/: {len(files)} items")
        if directory == 'data' and os.path.exists('data/cifar10'):
            cifar_items = os.listdir('data/cifar10')
            badnets_count = len([f for f in cifar_items if f.startswith('badnets')])
            print(f"      🦠 Poisoned datasets: {badnets_count}")
    else:
        print(f"   ❌ {directory}/: Not found")

# Results analysis (if available)
if os.path.exists(test_log_file):
    results_df = pd.read_csv(test_log_file, sep='\t')
    results_df['asr'] = pd.to_numeric(results_df['asr'], errors='coerce')
    
    print(f"\n📈 Cross-Intensity Test Results:")
    print(f"   📊 Total tests: {len(results_df)}")
    print(f"   📈 Max ASR: {results_df['asr'].max():.3f}")
    print(f"   📉 Min ASR: {results_df['asr'].min():.3f}")
    print(f"   📊 Mean ASR: {results_df['asr'].mean():.3f} ± {results_df['asr'].std():.3f}")

# Key research insights
print(f"\n🔬 Key Research Insights:")
print(f"   🎯 Training-inference intensity mismatch significantly affects ASR")
print(f"   📊 10x10 intensity matrix reveals robustness patterns")
print(f"   🛡️  Defense effectiveness varies with trigger intensity")
print(f"   🎭 Mixed intensity training shows different behavior")

print(f"\n📚 Citation:")
print(f"@inproceedings{{lin2025titim,")
print(f"    title = {{Revisiting Training-Inference Trigger Intensity in Backdoor Attacks}},")
print(f"    author = {{Lin, Chenhao and Zhao, Chenyang and Wang, Shiwei and Wang, Longtian and Shen, Chao and Zhao, Zhengyu}},")
print(f"    booktitle = {{34th USENIX Security Symposium (USENIX Security 25)}},")
print(f"    year = {{2025}}")
print(f"}}")

print(f"\n🎉 TITIM Experiment Complete! 🎉")
print(f"⏰ Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


# Final Documentation and Next Steps

## 🎯 Experiment Results & Next Steps

### 📊 What This Experiment Reveals
1. **Training-Inference Intensity Mismatch Effect**: How backdoor attack success rates vary when models are trained on one trigger intensity but tested on another.  
2. **Robustness Patterns**: The 10×10 intensity matrix reveals which intensity combinations are most/least effective.  
3. **Defense Vulnerabilities**: Different defense methods show varying effectiveness across intensity ranges.  
4. **Mixed Intensity Behavior**: Models trained with mixed intensities exhibit different robustness characteristics.

### 🔬 Research Implications
- **For Attackers**: Understanding intensity mismatch helps design more robust backdoors  
- **For Defenders**: Reveals new attack vectors and defense evaluation criteria  
- **For Researchers**: Provides framework for systematic backdoor robustness analysis

### 🚀 Next Steps
1. **Extend to Other Attacks**: Test with Blended, WaNet, SIG attacks  
2. **More Datasets**: Experiment with GTSRB, ImageNet, CelebA  
3. **Advanced Defenses**: Test more sophisticated defense methods  
4. **Real-world Scenarios**: Investigate practical intensity variations

### 📁 Generated Artifacts
- `data/`: Clean and poisoned datasets  
- `res/`: Trained backdoor models  
- `logs/`: Experimental results (TSV format)  
- `heatmaps/`: Visualization outputs (PDF format)

**🎉 Congratulations! You've completed the full TITIM experiment pipeline!**
